In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from cryptography.fernet import Fernet

In [ ]:
headers = {'User-Agent': 'Mozilla/5.0'}
base_url = "http://books.toscrape.com/catalogue/page-{}.html"
book_data = []

for page in range(1, 3):  # Scrape first 2 pages
    try:
        response = requests.get(base_url.format(page), headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        books = soup.select('article.product_pod')
        for book in books:
            title = book.h3.a['title']
            price = book.select_one('.price_color').text
            rating_tag = book.select_one('p.star-rating')
            rating = rating_tag['class'][1] if rating_tag else 'No rating'
            book_data.append({'Title': title, 'Price': price, 'Rating': rating})
        time.sleep(1)  # Respectful delay
    except Exception as e:
        print(f"Error on page {page}: {e}")

In [4]:
df = pd.DataFrame(book_data)
df.to_csv('books.csv', index=False, encoding='utf-8')
df.head()

,Title,Price,Rating
0,A Light in the Attic,Â£51.77,Three
1,Tipping the Velvet,Â£53.74,One
2,Soumission,Â£50.10,One
3,Sharp Objects,Â£47.82,Four
4,Sapiens: A Brief History of Humankind,Â£54.23,Five


In [5]:
key = Fernet.generate_key()
with open('secret.key', 'wb') as f:
    f.write(key)

In [6]:
with open('secret.key', 'rb') as f:
    key = f.read()

fernet = Fernet(key)

with open('books.csv', 'rb') as file:
    original = file.read()

encrypted = fernet.encrypt(original)
with open('books_encrypted.bin', 'wb') as enc_file:
    enc_file.write(encrypted)

In [7]:
with open('books_encrypted.bin', 'rb') as enc_file:
    encrypted_data = enc_file.read()

decrypted = fernet.decrypt(encrypted_data)
with open('books_decrypted.csv', 'wb') as dec_file:
    dec_file.write(decrypted)

# Preview decrypted content
pd.read_csv('books_decrypted.csv').head()

,Title,Price,Rating
0,A Light in the Attic,Â£51.77,Three
1,Tipping the Velvet,Â£53.74,One
2,Soumission,Â£50.10,One
3,Sharp Objects,Â£47.82,Four
4,Sapiens: A Brief History of Humankind,Â£54.23,Five
